<a href="https://colab.research.google.com/github/KenichiQaz/FacialRecognitionAttendance/blob/main/FaceRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fastai - Facial recognition with transformations

## Getting data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# copy the images from a google drive folder and unzip for use
!unzip '/content/drive/MyDrive/Colab Notebooks/data.zip' -d '/content/data/'

## Building the model

In [ ]:
# Import fast.ai for use in task
! [ -e /content ] && pip install -Uqq fastbook opendatasets
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
# Set the path for the image data
path='/content/data'

# Get path and remove corrupted images
TestImages = get_image_files(path)
verify = verify_images(TestImages)

# Open an image to see data
im=Image.open(TestImages[25])
with im: display(im.to_thumb(128,128))

In [ ]:
# Build datablock for training and validation
faces=DataBlock(
    blocks=(ImageBlock,CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2,seed=42),
    get_y=parent_label,
    item_tfms=Resize(128,ResizeMethod.Pad,pad_mode='zeros')
)

In [ ]:
# Load images from folders
dls=faces.dataloaders(path)

# After training validation split show some of the images
dls.train.show_batch(max_n=4,nrows=1)

In [ ]:
# Transform images and load them into training data too
faces=faces.new(item_tfms=Resize(128),batch_tfms=aug_transforms(mult=2))
dls=faces.dataloaders(path)
dls.train.show_batch(max_n=8,nrows=2,unique=True)

In [ ]:
# Transform images and load them into training data too
faces==faces.new(item_tfms=RandomResizedCrop(224,min_scale=0.5),
                 batch_tfms=aug_transforms()
                 )
dls=faces.dataloaders(path)
dls.train.show_batch(max_n=8,nrows=2,unique=True)

In [ ]:
# Show some of the validation set
dls.valid.show_batch(max_n=8,nrows=2,unique=True)

In [ ]:
# Select the model and finetune it on the new data
learn=vision_learner(dls, resnet50, metrics=accuracy)
learn.fine_tune(15)

In [ ]:
# Display confusion matrix
# interp=ClassificationInterpretation.from_learner(learn)
# interp.plot_confusion_matrix(figsize=(10,10))

In [ ]:
# Export the model
learn.export()

## Load model and predict name

In [ ]:
# Confirm model exists
path = Path()
path.ls(file_exts='.pkl')

In [ ]:
# Load model for inferrence
learn_inf=load_learner(path/'export.pkl')

In [ ]:
# Create upload button to test on unseen image
btn_upload = widgets.FileUpload()
btn_upload

In [ ]:
# Show uploaded image
img = PILImage.create(btn_upload.data[-1])

out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
out_pl

In [ ]:
# Take uploaded image and predict the name if the certainty is below 90% label it as unknown
pred,pred_idx,probs = learn_inf.predict(img)

lbl_pred = widgets.Label()
if probs[pred_idx] > 0.90:
  lbl_pred.value = f'Prediction: {pred};      Probability: {probs[pred_idx]:.04f}'
else:
  lbl_pred.value = f'Unknown.      Prediction: {pred};      Probability: {probs[pred_idx]:.04f}'
lbl_pred

## Mark attendance in a Google sheet